In [201]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

In [ ]:
pip install --upgrade tensorflow

In [ ]:
#GAN

z=tf.Variable(np.random.rand(1,3),dtype=tf.float32)

x=tf.Variable(np.random.rand(1,10),dtype=tf.float32)

def D():
    inp=tf.keras.Input(shape=(10,))
    x=tf.keras.layers.Dense(3)(inp)
    y=tf.keras.layers.Dense(1,activation='sigmoid')(x)
    model=tf.keras.Model(inputs=inp,outputs=y)
    return model


def G():
    inp=tf.keras.Input(shape=(3,))
    x=tf.keras.layers.Dense(3,activation='relu')(inp)
    y=tf.keras.layers.Dense(10)(x)
    model=tf.keras.Model(inputs=inp,outputs=y)
    return model


def train():
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    opt = tf.keras.optimizers.Adam(learning_rate=1)
    print("z=",z)
    no_of_epochs=100
    k_steps=10
    for i in range(no_of_epochs):
        with tf.GradientTape( persistent=True, watch_accessed_variables=True) as tap:

            real_op=model_d(x)
            real_loss=cross_entropy(tf.ones_like(real_op),real_op)
            fake_op=model_d(model_g(z))
            fake_loss=cross_entropy(tf.zeros_like(fake_op),fake_op)
            loss_d=real_loss+fake_loss
            if i%k_steps==0:
                loss_g=cross_entropy(tf.ones_like(fake_op),fake_op)
                #loss_d=tf.math.log(model_d(x))+tf.math.log(1-model_d(model_g(z)))

        var_list_d=model_d.trainable_weights
        #var_list.extend(model.trainable_weights)

        gradients=tap.gradient(loss_d,var_list_d)

        #print("grad=",gradients)

        if i%k_steps==0:
            var_list_g=[z]
            #var_list_g.extend(model_g.trainable_weights)


            gradients=tap.gradient(loss_g,var_list_g)

            print("grad=",gradients)

            opt.apply_gradients(zip(gradients, var_list_g))


            print("z=",z)
            
            
train()

In [ ]:
#Deep Q Learning

class Env:
    
    def __init__(self,state_dim,action_dim):
        self.state_dim=state_dim
        self.action_dim=action_dim
        self.model=self.model(state_dim,action_dim)
        
    def step(self,state,action):
        act=np.zeros((state.shape[0],self.action_dim))
        
        i=np.arange(0,state.shape[0])
       
        act[i,action]=1.0
       
        return self.model.predict([state,act])
        
    def model(self,state_dim,action_dim):
        state=tf.keras.Input(shape=(state_dim,))
        action=tf.keras.Input(shape=(action_dim,))
        conc=tf.keras.layers.Concatenate(axis=1)([state, action])
        
        x=tf.keras.layers.Dense(3,activation='relu')(conc)
        next_state=tf.keras.layers.Dense(state_dim,activation='sigmoid')(x)
        reward=tf.keras.layers.Dense(1)(x)
        model=tf.keras.Model(inputs=[state,action],outputs=[next_state,reward])
        return model

    

def Q(state_dim,action_dim):
    inp=tf.keras.Input(shape=(state_dim,))
    x=tf.keras.layers.Dense(3,activation='relu')(inp)
    y=tf.keras.layers.Dense(action_dim,activation='sigmoid')(x)
    model=tf.keras.Model(inputs=inp,outputs=y)
    return model


state_dim=2
action_dim=3
q1=Q(state_dim,action_dim)
q=Q(state_dim,action_dim)

env=Env(2,3)
state=np.random.rand(1,2)
action=[1]

episodes=100
k_steps=10
opt = tf.keras.optimizers.Adam(learning_rate=1)
    

for eps in range(episodes):
    (next_state,reward)=env.step(state,action)
    
    with tf.GradientTape(persistent=True, watch_accessed_variables=True) as tap:
        
        #print(next_state,reward)
        predicted=q(state)

        target=reward+tf.math.reduce_max(q1(next_state),axis=1) 


        loss=tf.keras.losses.MSE(target,predicted)
    
        #print("episodes =",eps)
        if eps%k_steps==0:
            #print("weights updated")
            q1.set_weights(q.get_weights()) 

        #print("target=",target," pred=",predicted)
        
        state=next_state
    gradients=tap.gradient(loss,q.trainable_weights)

    

    opt.apply_gradients(zip(gradients, q.trainable_weights))
    
    #print("grad applied")




In [133]:
# actor critic models




class Env:
    
    def __init__(self,state_dim,action_dim):
        self.state_dim=state_dim
        self.action_dim=action_dim
        #self.model=self.model(state_dim,action_dim)
        
    def step(self,state,action):
        return (np.random.rand(1,self.state_dim),np.random.rand(1)*10,True)
        
        
#     def model(self,state_dim,action_dim):
#         state=tf.keras.Input(shape=(state_dim,))
#         action=tf.keras.Input(shape=(action_dim,))
#         conc=tf.keras.layers.Concatenate(axis=1)([state, action])
        
#         x=tf.keras.layers.Dense(3,activation='relu',trainable=False)(conc)
#         next_state=tf.keras.layers.Dense(state_dim,activation='sigmoid',trainable=False)(x)
#         reward=tf.keras.layers.Dense(1,trainable=False)(x)
#         done=tf.keras.layers.Dense(1,activation='sigmoid',trainable=False)(x)
#         model=tf.keras.Model(inputs=[state,action],outputs=[next_state,reward,done])
#         return model

    def reset(self):
        return np.random.rand(1,self.state_dim)


    

def policy(state_dim,action_dim):
    inp=tf.keras.Input(shape=(state_dim,))
    x=tf.keras.layers.Dense(3,activation='relu')(inp)
    mu=tf.keras.layers.Dense(action_dim,activation='sigmoid')(x)
    sigma=tf.keras.layers.Dense(action_dim,activation='sigmoid')(x)
    
    dist=tfp.layers.DistributionLambda(lambda y: tfp.distributions.Normal(loc=y[0], scale=y[1]))([mu,sigma])
    
    model=tf.keras.Model(inputs=inp,outputs=dist)
    return model


def critic(state_dim):
    inp=tf.keras.Input(shape=(state_dim,))
    x=tf.keras.layers.Dense(3,activation='relu')(inp)
    y=tf.keras.layers.Dense(1)(x)
    model=tf.keras.Model(inputs=inp,outputs=y)
    return model



def run_episode(max_step):
    action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    values=tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    rewards=tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    state=env.reset()
    
    for t in range(max_step):

        dist=p(state)
        
        action_sample=tf.squeeze(dist.sample([1]),axis=0)
    
        (next_state,reward,done)=env.step(state,action_sample)
        
        action_probs=action_probs.write(t,tf.squeeze(dist.prob(action_sample)))
        values=values.write(t,tf.squeeze(v(state)))
        
        rewards=rewards.write(t,reward)
       
        state=next_state
   
    return (action_probs.stack(),values.stack(),rewards.stack())
        
def compute_loss(action_probs,values,expected_returns):
    
    action_logs=tf.math.log(action_probs)
    advantage=expected_returns-values
    actor_loss=-tf.reduce_sum(action_logs*advantage)
    critic_loss=tf.reduce_sum(tf.keras.losses.MSE(expected_returns,values))
    
    return (actor_loss,critic_loss)


def expected_return(rewards,gamma,standarize):
    n = tf.shape(rewards)[0]
    returns = tf.TensorArray(dtype=tf.float32, size=n)

    discounted_sum=tf.constant(0.0)
    for i in range(n):
        discounted_sum=rewards[i]+gamma*discounted_sum
        
        returns=returns.write(i,discounted_sum)
    returns=returns.stack()[::-1]      
    if standarize:
        returns = ((returns - tf.math.reduce_mean(returns)) /(tf.math.reduce_std(returns) + 0.00001))

    return returns    


def train_step(opt,max_step,gamma):
   
       
    with tf.GradientTape(persistent=True) as tap :
       
        (action_probs,values,rewards)=run_episode(max_step)
        returns=expected_return(rewards,gamma,True)
    
        action_probs, values, returns = [
            tf.expand_dims(x, 1) for x in [action_probs, values, returns]] 
 
        print(action_probs,values,rewards)
        (actor_loss,critic_loss)=compute_loss(action_probs,values,returns)
    print(actor_loss,critic_loss)
    
    p_grads=tap.gradient(actor_loss,p.trainable_weights)
    v_grads=tap.gradient(critic_loss,v.trainable_weights)
    
    print("p grads=",p_grads)
    print("v grads=",v_grads)
    opt.apply_gradients(zip(p_grads,p.trainable_weights))
    
    opt.apply_gradients(zip(v_grads,v.trainable_weights))


def train():

    episodes=1
    #k_steps=10
    opt = tf.keras.optimizers.Adam(learning_rate=1)

    gamma=0.1
    max_step=10
     
    for i in range(episodes):
        train_step(opt,max_step,gamma)


        
state_dim=2
action_dim=3

env=Env(state_dim,action_dim)

p=policy(state_dim,action_dim)

v=critic(state_dim)

train()

tf.Tensor(
[[[0.17339589 0.6973886  0.8148903 ]]

 [[0.82217306 0.10814424 0.82531536]]

 [[0.81147045 0.04377539 0.4597268 ]]

 [[0.6850138  0.07223559 0.8211778 ]]

 [[0.7916062  0.7285566  0.5754579 ]]

 [[0.70390385 0.6316571  0.8129708 ]]

 [[0.8057768  0.36423722 0.8024985 ]]

 [[0.79037565 0.5037587  0.7234709 ]]

 [[0.46806952 0.73017585 0.6679684 ]]

 [[0.80407786 0.7896915  0.5878836 ]]], shape=(10, 1, 3), dtype=float32) tf.Tensor(
[[ 0.16754758]
 [ 0.23071122]
 [ 0.23336972]
 [ 0.22892481]
 [-0.02624834]
 [ 0.11573507]
 [ 0.1117183 ]
 [ 0.10573964]
 [ 0.13343468]
 [ 0.07844465]], shape=(10, 1), dtype=float32) tf.Tensor(
[[0.6013521]
 [8.575658 ]
 [3.2821965]
 [1.4999222]
 [8.263745 ]
 [8.050822 ]
 [1.5405911]
 [1.6062477]
 [0.9716675]
 [3.1337337]], shape=(10, 1), dtype=float32)
tf.Tensor(-75.30272, shape=(), dtype=float32) tf.Tensor(102.49807, shape=(), dtype=float32)
p grads= [<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-8.7669525,  0.       ,  0.       ],
    

In [95]:

env=Env(2,3)
(next_state,reward)=env.step(np.random.rand(1,2),)

TypeError: step() missing 1 required positional argument: 'action'

In [55]:
action_advantage.shape

TensorShape([1, 1])

In [35]:


with tf.GradientTape(persistent=True, watch_accessed_variables=True) as tap:
    x=tf.Variable(np.array([2,3]),dtype=tf.float32)
    y=x*x

gradients=tap.gradient(y,[x])
print(gradients)
opt = tf.keras.optimizers.SGD(learning_rate=1)

opt.apply_gradients(zip(gradients,[x]))
    
print(x)

[<tf.Tensor: shape=(2,), dtype=float32, numpy=array([4., 6.], dtype=float32)>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([6., 9.], dtype=float32)>


In [39]:
tf.keras.losses.MSE(np.array([4]),np.array([3]))

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=int64, numpy=1>>

In [74]:
tf.Variable(np.array([[2,3],[1,3]]))[::-1]

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[3, 1],
       [3, 2]])>

In [87]:
dist=p(np.random.rand(1,3))

        
action_sample=tf.squeeze(dist.sample([1]),axis=0)

dist.prob(action_sample)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.22900645, 0.12034588]], dtype=float32)>

In [200]:

class CustomLayer (tf.keras.layers.Layer):
    
    def __init__(self,num):
        super(CustomLayer,self).__init__()
        self.num=num
        
    def build(self,input_shape):
        self.wg=tf.Variable(np.ones((self.num,)),dtype=tf.float32)
        
    @tf.custom_gradient
    def custom_op1(self,x,w):
        
        result=w
        
        def custom_grad(dy):
            grad1 = dy*x*3.0
            grad2=dy*w*2.0
            return grad1,grad2
        
        return result, custom_grad

    def call(self,inp):
        #print("wg===",self.wg)
        return self.custom_op1(inp,self.wg)


layer=CustomLayer(2)     
x=tf.Variable(np.array([1.0,2.0]),dtype=tf.float32)
opt = tf.keras.optimizers.Adam(learning_rate=1)
with tf.GradientTape(persistent=True) as tap:
    
    tap.watch(x)
    tap.watch(layer.trainable_weights)
    
    y=layer(x)
    
    print("y=",y)
    
    
var_list=[x]
var_list.extend(layer.trainable_weights)

grads=tap.gradient(y,var_list)


print(grads)
opt.apply_gradients(zip(grads,var_list))
    
    

y= tf.Tensor([1. 1.], shape=(2,), dtype=float32)
[<tf.Tensor: shape=(2,), dtype=float32, numpy=array([3., 6.], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 2.], dtype=float32)>]


<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [155]:
@tf.custom_gradient
def f(x,y):
    fx = x+y
    
    def grad(dy):
        return dy * 2,dy*3
    return fx, grad



x=tf.Variable(np.array([2.0]))
y=tf.Variable(np.array([2.0]))
with tf.GradientTape(persistent=True, watch_accessed_variables=True) as tap:
    
    p=f(x,y)
    q=f(p,y)

print(tap.gradient(q,x),tap.gradient(q,y))


tf.Tensor([4.], shape=(1,), dtype=float64) tf.Tensor([9.], shape=(1,), dtype=float64)


In [194]:
layer.trainable_weights

[<tf.Variable 'custom_layer_43/Variable:0' shape=(2,) dtype=float32, numpy=array([1., 1.], dtype=float32)>]

In [215]:
w1=np.random.rand(10,3)
x=np.random.rand(1,10)

In [216]:
np.dot(x,w1).shape

(1, 3)